In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from ase import Atoms
from ase.visualize import view
from ase.build import make_supercell
from ase.io import read

from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt

In [ ]:
from dotenv import load_dotenv
import os
from mp_api.client import MPRester
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor

load_dotenv()

API_KEY=os.getenv("MP_API_KEY")

def save_material_ids(output_file="material_ids.txt", max_results=100000, page_size=1000):
    with MPRester(API_KEY) as mpr, open(output_file, "w") as f:
        total_fetched = 0
        page = 0

        while total_fetched < max_results:
            results = mpr.materials.search(all_fields=False)

            if not results:
                break

            for entry in results:
                f.write(f"{entry.material_id},{entry.formula_pretty}\n")

            total_fetched += len(results)
            page += 1

        print(f"Saved {total_fetched} materials to {output_file}")

def get_atoms(id):
    with MPRester(API_KEY) as mpr:
        structure = mpr.get_structure_by_material_id(id)
        
        atoms = AseAtomsAdaptor.get_atoms(structure)
        
        return atoms
    
# save_material_ids()

Helper for converting atoms into respective model format (orb or mattersim)

In [ ]:
from orb_models.forcefield.atomic_system import ase_atoms_to_atom_graphs
from mattersim.datasets.utils.convertor import GraphConvertor

mattersim_converter = GraphConvertor("m3gnet", 5.0, True, 4.0)

def atoms_to_graph_orb(atoms):
    graph = ase_atoms_to_atom_graphs(atoms)
    senders = graph.senders
    receivers = graph.receivers
    edge_feats = graph.edge_features

    G = nx.Graph()
    G.add_nodes_from(range(graph.n_node))
    G.graph['edge_weight_attr'] = 'weight'
    
    min_dist = min(edge_feats['r'])

    for i, u in enumerate(senders):
        G.add_edge(u.item(), receivers[i].item(), weight=int(edge_feats['r'][i] / min_dist * 1000))

    return G 

def atoms_to_graph_mattersim(atoms):
    return to_networkx(mattersim_converter.convert(atoms, None, None, None))

Helpers for retrieving relevant metrics from partition

In [ ]:
def total_node_count(G, partitions, extended_partitions):
    """
    Returns the total number of nodes including overlapping nodes in extended
    partitions
    """
    
    return sum(len(x) for x in extended_partitions)

def root_node_count(G, partitions, extended_partitions):
    """
    Returns the total number of root nodes (the number of vertices in the
    original graph)
    """
    
    return sum(len(x) for x in partitions)

def extended_ratio(G, partitions, extended_partitions):
    """
    Returns the ratio of extended nodes to the total number of nodes (higher 
    ratio means more neighbors were captured in the partition, which we don't
    want)
    
    i.e. how many of our nodes are 'redundant'?
    """
    
    root = root_node_count(G, partitions, extended_partitions)
    total = total_node_count(G, partitions, extended_partitions)
    
    return (total - root) / (total)

def core_partition_stats(G, partitions, extended_partitions):
    """
    Returns statistics about the extended partitions (max, min, mean, std, 
    range)
    """
    
    partition_sizes = [len(p) for p in partitions]
    sizes = np.array(partition_sizes)
    
    return sizes.max(), sizes.min(), sizes.mean(), sizes.std(), (sizes.max() - sizes.mean())

def extended_partition_stats(G, partitions, extended_partitions):
    """
    Returns statistics about the extended partitions (max, min, mean, std, 
    range)
    """
    
    partition_sizes = [len(p) for p in extended_partitions]
    sizes = np.array(partition_sizes)
    
    return sizes.max(), sizes.min(), sizes.mean(), sizes.std(), (sizes.max() - sizes.mean())

def num_cut_edges(G, partitions, extended_partitions):
    """
    Returns the number of edges that were cut between partitions
    """
    
    node_to_part = {}
    for i, part in enumerate(partitions):
        for node in part:
            node_to_part[node] = i

    cut_edges = set()

    # Check each edge to see if it crosses partitions
    for u, v in G.edges():
        pu = node_to_part.get(u)
        pv = node_to_part.get(v)
        if pu is not None and pv is not None and pu != pv:
            cut_edges.add(tuple(sorted((u, v))))

    return len(cut_edges)

def sum_cut_edge_weights(G, partitions, extended_partitions):
    """
    For all cut edges, return sum(x) where x is the weight of each edge
    """
    
    cut_edges = set()
    node_to_partition = {node: i for i, part in enumerate(partitions) for node in part}
    for u, v in G.edges():
        if node_to_partition.get(u) != node_to_partition.get(v):
            cut_edges.add(tuple(sorted((u, v))))
    return sum(G.edges[u, v].get('weight', 0.0) for u, v in cut_edges)

def sum_inverse_cut_edge_weights(G, partitions, extended_partitions):
    """
    For all cut edges, return sum(1/x) where x is the weight of each edge
    
    Maybe minimizing this quantity will help because that means we are
    cutting fewer edges of atoms that are closer together (and thus
    more likely to have a greater impact on each other)
    """
    
    cut_edges = set()

    node_to_partition = {}
    for i, part in enumerate(partitions):
        for node in part:
            node_to_partition[node] = i

    for u, v in G.edges():
        p_u = node_to_partition.get(u)
        p_v = node_to_partition.get(v)
        if p_u is not None and p_v is not None and p_u != p_v:
            cut_edges.add(tuple(sorted((u, v))))

    # Sum inverse of edge weights for cut edges
    total_inverse_weight = 0.0
    for u, v in cut_edges:
        w = G.edges[u, v].get('weight', None)
        if w is not None and w > 0:
            total_inverse_weight += 1.0 / (w)

    return total_inverse_weight

Helper for running the benchmark and recording metrics

In [ ]:
from partitioning import part_spectral, part_metis, part_grid, part_metis_unweighted

def benchmark_trial_orb(atoms, distance=1, granularity=2):
    G = atoms_to_graph_orb(atoms)
    
    print(len(G.edges))

    extended_partition_results = []
    results = []

    for method_name, partition_fn in [
        ("metis", part_metis),
        ("spectral", part_spectral),
        ("grid", part_grid),
        ("metis_unweighted", part_metis_unweighted)
    ]:
        if method_name == 'spectral' and len(atoms) >= 10000:
            continue
        
        try:
            partitions, extended_partitions = partition_fn(atoms, G, granularity ** 3, distance=distance)

            # Core stats
            core_max, core_min, core_mean, core_std, core_range = core_partition_stats(G, partitions, extended_partitions)
            # Extended stats
            ext_max, ext_min, ext_mean, ext_std, ext_range = extended_partition_stats(G, partitions, extended_partitions)

            result = {
                'method': method_name,
                'node_count': total_node_count(G, partitions, extended_partitions),
                'root_node_count': root_node_count(G, partitions, extended_partitions),
                'extended_ratio': extended_ratio(G, partitions, extended_partitions),
                'cut_edges': num_cut_edges(G, partitions, extended_partitions),
                'cut_weight_sum': sum_cut_edge_weights(G, partitions, extended_partitions),
                'inverse_cut_weight_sum': sum_inverse_cut_edge_weights(G, partitions, extended_partitions),

                # Core stats
                'core_max': core_max,
                'core_min': core_min,
                'core_mean': core_mean,
                'core_std': core_std,
                'core_range': core_range,

                # Extended stats
                'ext_max': ext_max,
                'ext_min': ext_min,
                'ext_mean': ext_mean,
                'ext_std': ext_std,
                'ext_range': ext_range,
            }

            results.append(result)

            for part, ext_part in zip(partitions, extended_partitions):
                extended_partition_results.append({
                    'method': method_name,
                    'core_n': len(part),
                    'n': len(ext_part)
                })

        except Exception as e:
            print(f"[{method_name}] Partitioning failed: {e}")

    return results, extended_partition_results


## Run benchmark

Load material ids

In [ ]:
import csv

NUM_MATERIALS = 10
MATERIAL_IDS_FILE = 'material_ids.txt'

material_ids = []

with open(MATERIAL_IDS_FILE, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        material_ids.append(row[0])
        
material_ids = np.array(material_ids)

# choose a random subset
material_ids = np.random.choice(material_ids, size=NUM_MATERIALS)
material_ids

In [ ]:
from ase.io import write

FULL_DF_FILE = 'results/chunk_0_n_200000_full_mattersim.csv'
PARTITION_DF_FILE = 'results/chunk_0_n_200000_partitions_mattersim.csv'

all_df = pd.read_csv(FULL_DF_FILE)
partition_df = pd.read_csv(PARTITION_DF_FILE)

In [ ]:
partition_df

In [ ]:
summary = all_df.groupby("method")[[
    "core_max", "core_min", "core_mean",
    "core_std", "core_range"
]].mean().round(3)

print(summary.to_string(line_width=1000))
print('\n\n')

summary = all_df.groupby("method")[[
    "ext_max", "ext_min", "ext_mean",
    "ext_std", "ext_range"
]].mean().round(3)

print(summary.to_string(line_width=1000))

In [ ]:
methods = partition_df['method'].unique()
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']  # Add more if needed
bins = 10

plt.figure(figsize=(8, 6))

for method, color in zip(methods, colors):
    subset = partition_df[partition_df['method'] == method]['core_n']
    plt.hist(subset, bins=bins, alpha=0.6, label=method, color=color)

plt.title("Core Partition Size by Partitioning Method")
plt.xlabel("Core Partition Size")
plt.ylabel("Frequency")
plt.legend(title="Method")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
methods = partition_df['method'].unique()
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']  # Add more if needed
bins = 10

plt.figure(figsize=(8, 6))

for method, color in zip(methods, colors):
    subset = partition_df[partition_df['method'] == method]['n']
    plt.hist(subset, bins=bins, alpha=0.6, label=method, color=color)

plt.title("Extended Partition Size by Partitioning Method")
plt.xlabel("Extended Partition Size")
plt.ylabel("Frequency")
plt.legend(title="Method")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
methods = all_df['method'].unique()
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']  # Add more if needed
bins = 10

plt.figure(figsize=(8, 6))

for method, color in zip(methods, colors):
    subset = all_df[all_df['method'] == method]['extended_ratio']
    plt.hist(subset, bins=bins, alpha=0.6, label=method, color=color)

plt.title("Ratio of Extended Partition Size to All Atoms by Partitioning Method")
plt.xlabel("Extended Ratio")
plt.ylabel("Frequency")
plt.legend(title="Method")
plt.grid(True)
plt.tight_layout()
plt.show()